In [16]:
import pickle 
import pandas as pd 
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint


In [2]:
import wandb

wandb.login()



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [18]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'test_accuracy',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric



parameters_dict = {
    'optimizer': {
        'value': 'adam'
        },
    'f1_layer_size': {
        'values': [64,128, 256]
        },
    'f2_layer_size': {
        'values': [16,32,64,128]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'values': [1000,5000]},
    'validation_split': {
        'value': 0.1},
    'loss':{
        'value': 'CCL'
    }
    })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.0002
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

import pprint

pprint.pprint(sweep_config)



{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'test_accuracy'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [1000, 5000]},
                'f1_layer_size': {'values': [64, 128, 256]},
                'f2_layer_size': {'values': [16, 32, 64, 128]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0002,
                                  'min': 0.0001},
                'loss': {'value': 'CCL'},
                'optimizer': {'value': 'adam'},
                'validation_split': {'value': 0.1}}}


In [4]:
sweep_config

{'method': 'random',
 'metric': {'name': 'test_accuracy', 'goal': 'minimize'},
 'parameters': {'optimizer': {'value': 'adam'},
  'f1_layer_size': {'values': [64, 128, 256]},
  'f2_layer_size': {'values': [16, 32, 64, 128]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'epochs': {'values': [1000, 5000]},
  'validation_split': {'value': 0.1},
  'loss': {'value': 'CCL'},
  'learning_rate': {'distribution': 'uniform', 'min': 0.0001, 'max': 0.0002},
  'batch_size': {'distribution': 'q_log_uniform_values',
   'q': 8,
   'min': 32,
   'max': 256}}}

In [5]:
sweep_id = wandb.sweep(sweep_config, project="grv-model-track-onehot")

Create sweep with ID: 50nrzbob
Sweep URL: https://wandb.ai/nickojelly/grv-model-track-onehot/sweeps/50nrzbob


In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
REBUILD_DATA = True
class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open( r"DATA/total_list.npy", 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    raceIDs,inputs,classes = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.classes[i]) == 8:
                self.training_data.append([np.array(self.inputs[i]),np.array(self.classes[i])])
            else:
                adjustedList = self.classes[i]+([8]*(8-len(self.classes[i])))
                self.training_data.append([np.array(self.inputs[i]),np.array(adjustedList)])
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)

if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()

c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [5]:
#dataset setup
training_data = grv.training_data

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])

my_dataset = TensorDataset(X,Y_w) 
my_dataloader = DataLoader(my_dataset)

C:\Users\Nick\AppData\Local\Temp\ipykernel_20864\2789279249.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  X = torch.Tensor([i[0] for i in training_data])


In [6]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.0001,
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))

config["dataset_size"]

67233

In [7]:
#have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
	dataset_size = len(dataset)
	indices = list(range(dataset_size))
	split = int(np.floor(config["validation_split"] * dataset_size))
	random_seed= 42
	np.random.seed(random_seed)
	np.random.shuffle(indices)
	train_indices, val_indices = indices[split:], indices[:split]

	if train:
		dataset_sampler = SubsetRandomSampler(train_indices)
		loader = torch.utils.data.DataLoader(dataset=dataset,
										batch_size=config["batch_size"], 
										shuffle=False,
										pin_memory=True, num_workers=2, sampler = dataset_sampler)
	else:
		dataset_sampler  = SubsetRandomSampler(val_indices)
		loader = torch.utils.data.DataLoader(dataset=dataset,
								shuffle=False,
								pin_memory=True, num_workers=2, sampler = dataset_sampler)


	
	return loader

							

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(81, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return nn.Softmax(x, dim=1)
    

In [9]:
def build_network(f1_layer_size,f2_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Linear(96, f1_layer_size), nn.ReLU(),
        nn.Linear(f1_layer_size,f2_layer_size), nn.ReLU(),
        nn.Linear(f2_layer_size, 8), nn.Softmax())
        

    return network

In [10]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset,config, train=True)
    test_loader = make_loader(dataset,config, train=False)
    # Make the model
    #model = Net().to(device)
    model = build_network(config['f1_layer_size'],config['f2_layer_size'],config['dropout'])

    # Make the loss and optimizer
    criterion = nn.HuberLoss(reduction='mean', delta=1.0)
    #criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()

    optimizer = config["optimizer"]

    if optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(),
                              lr=config["learning_rate"], momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=config["learning_rate"])
    #optimizer = torch.optim.Adam(
    #    model.parameters(), lr=config["learning_rate"])
    
    return model, train_loader, test_loader, criterion, optimizer

In [20]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total,max_sum, max_w_sum = 0, 0,0,0
        pred = []
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            #print(f"{images=},\n,{labels=}")
            max, predicted = torch.max(outputs.data, 1)
            _,real = torch.max(labels.data,1)
            pred.append(predicted.item())
            #print(f"{real.item(),predicted.item()=}")
            total += labels.size(0)
            correct += (predicted.item() == real.item())
            if (predicted.item() == real.item()):
                max_sum += max
            else:
                max_w_sum += max

            #print(f"{correct=}")

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        
        wandb.log({"test_accuracy": correct / total, "correct_conf": max_sum/correct, "incorrect_conf":(max_w_sum)/(total-correct)})
        wandb.log({'predlist':pred})
        table = wandb.Table(data=pred, columns=["pred_class"])
        wandb.log({'my_histogram': wandb.plot.histogram(table, "pred_class",
 	            title="Prediction Score Distribution")})
    # Save the model in the exchangeable ONNX format
    #torch.onnx.export(model, images, "model.onnx")
    #wandb.save("model.onnx")


In [12]:
def train(model, loader,test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 250) == 0:
                train_log(loss, example_ct, epoch)

        if epoch %5 ==0:
            test(model,test_loader)

def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [13]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    #print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [14]:
def model_pipeline(config=None):
    dataset = my_dataset
    # tell wandb to get started
    with wandb.init(project="pytorch-demo-sweep", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config, dataset)
      model = model.to(device)
      print(model)

      # and use them to train the model
      train(model, train_loader,test_loader, criterion, optimizer, config)

      # and test its final performance
      test(model, test_loader)

    return model

In [22]:
sweep_config

{'method': 'random',
 'metric': {'name': 'test_accuracy', 'goal': 'minimize'},
 'parameters': {'optimizer': {'value': 'adam'},
  'f1_layer_size': {'values': [64, 128, 256]},
  'f2_layer_size': {'values': [16, 32, 64, 128]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'epochs': {'values': [1000, 5000]},
  'validation_split': {'value': 0.1},
  'loss': {'value': 'CCL'},
  'learning_rate': {'distribution': 'uniform', 'min': 0.0001, 'max': 0.0002},
  'batch_size': {'distribution': 'q_log_uniform_values',
   'q': 8,
   'min': 32,
   'max': 256}}}

In [21]:
normal_config = dict(
    epochs=500,
    classes=8,
    f1_layer_size=128,
    f2_layer_size=32,
    batch_size=128,
    learning_rate=0.0001,
    optimizer = 'adam',
    loss = 'Huber',
    dropout = 0.2, 
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))

model = model_pipeline(config=normal_config)

In [19]:
#model_pipeline(config)

CUDA_LAUNCH_BLOCKING=1


wandb.agent(sweep_id, function=model_pipeline, count=1)

wandb: Agent Starting Run: rk2h2dmj with config:
wandb: 	batch_size: 224
wandb: 	dropout: 0.5
wandb: 	epochs: 5000
wandb: 	f1_layer_size: 128
wandb: 	f2_layer_size: 128
wandb: 	learning_rate: 0.00012891942494083511
wandb: 	loss: CCL
wandb: 	optimizer: adam
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/5000 [00:00<?, ?it/s]

{'batch_size': 224, 'dropout': 0.5, 'epochs': 5000, 'f1_layer_size': 128, 'f2_layer_size': 128, 'learning_rate': 0.00012891942494083511, 'loss': 'CCL', 'optimizer': 'adam', 'validation_split': 0.1}
5000
{'batch_size': 224, 'dropout': 0.5, 'epochs': 5000, 'f1_layer_size': 128, 'f2_layer_size': 128, 'learning_rate': 0.00012891942494083511, 'loss': 'CCL', 'optimizer': 'adam', 'validation_split': 0.1}
Sequential(
  (0): Linear(in_features=96, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=8, bias=True)
  (5): Softmax(dim=None)
)


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  0%|          | 1/5000 [00:19<26:49:52, 19.32s/it]

Accuracy of the model on the 6723 test images: 15.39491298527443%


...

100%|█████████▉| 4996/5000 [5:57:28<00:24,  6.08s/it]

Accuracy of the model on the 6723 test images: 23.218801130447716%


100%|██████████| 5000/5000 [5:57:35<00:00,  4.29s/it]


Accuracy of the model on the 6723 test images: 23.322921314889186%



correct_conf,▁▄▄▅▅▆▆▇▇▇▇▇▇▇▇█████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
incorrect_conf,▁▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇████████████████████████
loss,█▆▇▆▅▇▄▅▄▃▅▄▄▄▄▁▃▃▅▁▃▂▄▂▂▂▂▃▂▁▂▃▄▂▂▂▃▃▃▁
test_accuracy,▁▇█▅▆▆▄▄▄▄▄▄▅▅▄▄▆▅▅▅▄▄▃▃▄▃▄▅▄▄▃▄▄▃▃▄▄▃▂▃
correct_conf,0.9843
epoch,4999
incorrect_conf,0.97784
loss,1.83368
test_accuracy,0.23323


In [20]:
import wandb
sweep_configuration = {
    "name": "my-awesome-sweep",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "grid",
    "parameters": {
        "a": {
            "values": [1, 2, 3, 4]
        }
    }
}

def my_train_func():
    # read the current value of parameter "a" from wandb.config
    wandb.init()
    a = wandb.config.a

    wandb.log({"a": a, "accuracy": a + 1})

sweep_id = wandb.sweep(sweep_configuration)

# run the sweep
wandb.agent(sweep_id, function=my_train_func)

Create sweep with ID: uybi9eba
Sweep URL: https://wandb.ai/nickojelly/grv-model-track-onehot/sweeps/uybi9eba


wandb: Agent Starting Run: kbvqgn0f with config:
wandb: 	a: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


a,▁
accuracy,▁
a,1
accuracy,2


wandb: Agent Starting Run: 23r6vfth with config:
wandb: 	a: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


a,▁
accuracy,▁
a,2
accuracy,3


wandb: Agent Starting Run: xi7zi1xj with config:
wandb: 	a: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


a,▁
accuracy,▁
a,3
accuracy,4


wandb: Agent Starting Run: 2sbnc08v with config:
wandb: 	a: 4
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


a,▁
accuracy,▁
a,4
accuracy,5


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [21]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[81,256,32,8],
    batch_size=128,
    learning_rate=0.0001,
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))

config["dataset_size"]

model = model_pipeline(config, my_dataset)

TypeError: model_pipeline() takes from 0 to 1 positional arguments but 2 were given

In [ ]:
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
Y_w = torch.tensor([i for i in Y_w])
Y_w[1:10]

In [ ]:
#indexed win only
Y_wi = []
for i in Y:
    #print(torch.argmin(i))
    Y_wi.append(torch.argmin(i))
Y_w = torch.tensor(Y_wi)
print(Y_w)

In [ ]:
#Generates places
Y_p = []
for i in Y:
    place = [x*x for x in i]
    Y_p.append(place)
    
Y_p[1]

In [ ]:
#validating model is new
for (i,j) in net.named_parameters():
    print(i,j)

In [ ]:
winners = [i.item() for i in Y_wi]
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
    
List = winners
print(most_frequent(List))